In [ ]:
!pip install pysentimiento
!pip install emoji
!pip install -U pycld3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 KB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=05f0f871651924ae6248bad406d629f67c496f43534d95cd4dfcaac4ae69ba03
  Stored in directory: /root/.cache/pip/wheels/5e/8c/80/c3646df8201

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
import os
import cld3
import pandas as pd
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

os.listdir('/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Twitter') # Listagem dos arquivos do twitter

['twitter_all.csv',
 'twitter_all_limpo.csv',
 'df_AR.csv',
 'df_EC.csv',
 'df_CL.csv',
 'df_BR.csv',
 'df_MX.csv',
 'df_ES.csv',
 'Classificados',
 '.ipynb_checkpoints']

# Dataset não rotulado

In [ ]:
def lang_detector(x): # Função simples para a detecção da lingua no arquivo
  try:
    return cld3.get_language(x)
  except:
    return '?'

# Instanciar os dois modelos de analise de sentimento

analyzer_es = create_analyzer(task="sentiment", lang="es")
analyzer_en = create_analyzer(task="sentiment", lang="en")
file_list = os.listdir('/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Reddit') # Lista de arquivos do reddit

for file in file_list:
  # Com a estrutura que montamos, um arquivo terminado em:
  # _local refere-se aos subreddits especificos daqueles países, majoritariamente em lingua espanhola
  # _global refere-se ao subreddit de covid internacionalizado, majoritariamente em lingua inglesa
  # _multilanguage refere-se aos datasets unificados, podem possuir qualquer lingua.
  if '_local' in file:
    df = pd.read_csv(f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Reddit/{file}')
    df.dropna(inplace = True)
    df.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df['lang'] = df['body'].apply(lambda x: lang_detector(x)) # Detecção de idioma, no dataset _local não é realmente necessária
    df['lang'] = df['lang'].apply(lambda x: x[0])
    df = df.drop(df[(df['author'] == 'AutoModerator') | (df['author'] == '[deleted]')].index) # Exclusão de comentários de bots de moderação e de autores excluídos
    df.drop_duplicates(subset='body', keep='first', inplace=True)
    
    df['body'] = df['body'].apply(lambda x: preprocess_tweet(x, lang = 'es')) # Pre processamento dos textos
    df['pred'] = df['body'].apply(lambda x: analyzer_es.predict(x))  # Predição do sentimento
    df['label'] = df['pred'].apply(lambda x: x.output)
    df['probas'] = df['pred'].apply(lambda x: x.probas)

    df.to_csv(path_or_buf = f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Reddit/Classificados/{file[:-4]}_labelled.csv', sep = ',')

  # A mesma estrutura de predição repete-se para os outros datasets.
  if '_global' in file:
    df = pd.read_csv(f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Reddit/{file}')
    df.dropna(inplace = True)
    df.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df['lang'] = df['body'].apply(lambda x: lang_detector(x)) # Detecção de idioma, no dataset _global não é realmente necessária
    df['lang'] = df['lang'].apply(lambda x: x[0])
    df = df.drop(df[(df['author'] == 'AutoModerator') | (df['author'] == '[deleted]')].index)
    df.drop_duplicates(subset='body', keep='first', inplace=True)
    
    df['body'] = df['body'].apply(lambda x: preprocess_tweet(x, lang = 'en'))
    df['pred'] = df['body'].apply(lambda x: analyzer_en.predict(x))
    df['label'] = df['pred'].apply(lambda x: x.output)
    df['probas'] = df['pred'].apply(lambda x: x.probas)

    df.to_csv(path_or_buf = f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Reddit/Classificados/{file[:-4]}_labelled.csv', sep = ',')

  if '_multilanguage' in file:
    df = pd.read_csv(f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Reddit/{file}')
    df.dropna(inplace = True)
    df.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df['lang'] = df['body'].apply(lambda x: lang_detector(x)) # Detecção de idioma, no dataset _multilanguage serve para aplicar determinados modelos em determinadas linguas
    df['lang'] = df['lang'].apply(lambda x: x[0])
    df = df.drop(df[(df['author'] == 'AutoModerator') | (df['author'] == '[deleted]')].index)
    df.drop_duplicates(subset='body', keep='first', inplace=True)
    
    # Os datasets são filtrados e classificados com o mesmo fluxo
    df1 = df[df['lang'] == 'en'].copy()
    df1['body'] = df1['body'].apply(lambda x: preprocess_tweet(x, lang = 'en'))
    df1['pred'] = df1['body'].apply(lambda x: analyzer_en.predict(x))
    df1['label'] = df1['pred'].apply(lambda x: x.output)
    df1['probas'] = df1['pred'].apply(lambda x: x.probas)

    df2 = df[df['lang'] == 'es'].copy()
    df2['body'] = df2['body'].apply(lambda x: preprocess_tweet(x, lang = 'es'))
    df2['pred'] = df2['body'].apply(lambda x: analyzer_es.predict(x))
    df2['label'] = df2['pred'].apply(lambda x: x.output)
    df2['probas'] = df2['pred'].apply(lambda x: x.probas)

    df = pd.concat([df1, df2], ignore_index = True)
    df.to_csv(path_or_buf = f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Reddit/Classificados/{file[:-4]}_labelled.csv', sep = ',')


In [ ]:
# Classificação do dataset do twitter

def lang_detector(x):
  try:
    return cld3.get_language(x)
  except:
    return '?'

analyzer_es = create_analyzer(task="sentiment", lang="es")
analyzer_en = create_analyzer(task="sentiment", lang="en")
file_list = os.listdir('/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Twitter')

# No twitter os datasets estão separados por países, porém os tweets não possuem nenhuma distinção de lingua como no reddit.
for file in file_list:
  if 'df' in file:
    df = pd.read_csv(f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Twitter/{file}')
    df.dropna(inplace = True)
    df.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df['lang'] = df['text'].apply(lambda x: lang_detector(x)) # A detecção de linguagem é crucial para avaliramos apenas os tweets corretos
    df['lang'] = df['lang'].apply(lambda x: x[0])
    df.drop_duplicates(subset='text', keep='first', inplace=True)
    
    df1 = df[df['lang'] == 'en'].copy()
    df1['text'] = df1['text'].apply(lambda x: preprocess_tweet(x, lang = 'en'))
    df1['pred'] = df1['text'].apply(lambda x: analyzer_en.predict(x))
    df1['label'] = df1['pred'].apply(lambda x: x.output)
    df1['probas'] = df1['pred'].apply(lambda x: x.probas)

    df2 = df[df['lang'] == 'es'].copy()
    df2['text'] = df2['text'].apply(lambda x: preprocess_tweet(x, lang = 'es'))
    df2['pred'] = df2['text'].apply(lambda x: analyzer_es.predict(x))
    df2['label'] = df2['pred'].apply(lambda x: x.output)
    df2['probas'] = df2['pred'].apply(lambda x: x.probas)

    df = pd.concat([df1, df2], ignore_index = True)
    df.to_csv(path_or_buf = f'/content/gdrive/MyDrive/BTC-Dados-G1/Estrutura Medallion/Prata/Twitter/Classificados/{file[:-4]}_labelled.csv', sep = ',')

# Dataset rotulado (baseline)

In [ ]:
df_rotulado = pd.read_csv('/content/gdrive/MyDrive/COVID19_Tweets/amostra_rotulada.csv').drop(['Unnamed: 0'], axis = 1)
df_rotulado = df_rotulado.sample(5000)

In [ ]:
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

# Teste das métricas do modelo em um dataset já rotulado sobre covid
analyzer = create_analyzer(task="sentiment", lang="en")
df_rotulado['text'] = df_rotulado['text'].apply(lambda x: preprocess_tweet(x))
df_rotulado['pred'] = df_rotulado['text'].apply(lambda x: analyzer.predict(x))
df_rotulado['label'] = df_rotulado['pred'].apply(lambda x: x.output)
df_rotulado['probas'] = df_rotulado['pred'].apply(lambda x: x.probas)

In [ ]:
# Redefinição das labels para obtenção das métricas
df_rotulado['Sentiment_Label'] = df_rotulado['Sentiment_Label'].map({ 'neutral': 'NEU',
                                                                      'negative': 'NEG',
                                                                      'positive': 'POS'})
df_rotulado.head()

,Tweet_ID,Sentiment_Label,text,pred,label,probas
94694,1220399453158293505,NEU,RT @usuario: China allocates 1 billion yuan ($...,"AnalyzerOutput(output=NEU, probas={NEU: 0.938,...",NEU,"{'NEG': 0.0020287414081394672, 'NEU': 0.937936..."
128971,1220457095620976642,NEG,RT @usuario: 🤦 emoji tono de piel claro medio ...,"AnalyzerOutput(output=NEG, probas={NEG: 0.963,...",NEG,"{'NEG': 0.9630939364433289, 'NEU': 0.034001730..."
61230,1220347645606055936,NEU,RML Hospital In Delhi Sets Up First Corona Vir...,"AnalyzerOutput(output=NEU, probas={NEU: 0.978,...",NEU,"{'NEG': 0.0028543630614876747, 'NEU': 0.978310..."
67761,1220362114067464193,NEU,RT @usuario: China appears to be locking down ...,"AnalyzerOutput(output=NEU, probas={NEU: 0.949,...",NEU,"{'NEG': 0.042912572622299194, 'NEU': 0.9485344..."
7037,1220013579275055108,NEU,RT @usuario: WHO interim guidance on a range o...,"AnalyzerOutput(output=NEU, probas={NEU: 0.829,...",NEU,"{'NEG': 0.0020122884307056665, 'NEU': 0.828626..."


In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score

y_true = df_rotulado['Sentiment_Label']
y_pred = df_rotulado['label']

print(f'F1 score: {round(f1_score(y_true, y_pred, average = "macro"), 4)}')
print(f'Recall: {round(recall_score(y_true, y_pred, average = "weighted"), 4)}')
print(f'Precision: {round(precision_score(y_true, y_pred, average = "weighted"), 4)}')

F1 score: 0.6069
Recall: 0.6956
Precision: 0.7124
